In [2]:
# import libraries first
import pandas as pd 
import numpy as np # Numberical Python
import datetime as dt

In [2]:
rent = pd.read_excel('Moving annual rents by suburb September quarter 2018.xlsx', index_col=None, header=None, sheet_name='All properties') 
region = pd.read_csv('SSC_2016_AUST.csv', low_memory=False)
postcode = pd.read_csv('Australian_Post_Codes_Lat_Lon.csv', low_memory=False)
df = pd.read_excel('industry_1.xls', index_col=None, header=None)

In [3]:
sub_rent = list(rent.iloc[3:,1].str.upper().unique())
sub_rent2 = []
for k in sub_rent:
    aa = k.split('-')
    sub_rent2 += aa

In [5]:
vic_region = region[region['STATE_NAME_2016']== 'Victoria']
subs0 = list(vic_region['SSC_NAME_2016'].str.upper().unique())
subs = []
for k in subs0:
    a0 = k.split('(')
    subs.append(a0[0])

In [6]:
vic_loc = postcode[postcode['state']=='VIC']
vic_loc['suburb'] = vic_loc['suburb'].apply(lambda x: x.upper())
suburbs = list(vic_loc['suburb'].str.upper().unique())

C:\Users\YHChen\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
col_name = list(df.iloc[7,0:3].values)+ list(df.iloc[6,3:37].values)
df1 = df.iloc[8:19403,0:37]
df1.columns = col_name

In [8]:
df1['LABEL'] = df1['LABEL'].str.upper()
len(df1['LABEL'].unique())
list1 = list(df1['LABEL'].unique())
df2 =  pd.concat([df.iloc[1537:2004,0:37],df.iloc[7353:10587,0:37]])
df2.columns = col_name
df2['LABEL'] = df2['LABEL'].str.upper()
len(df2['LABEL'].unique())
list2 = list(df1['LABEL'].unique())

In [9]:
state = ['(WA)','(SA)','(NSW)','(QLD)','(TA)']
list3 = []
for j in list2:
    if j in subs or j in sub_rent2:
        list3.append(j)
    else:
        for k in subs:
            if k in j:
                if not any(s in j for s in state):
                        list3.append(j)
                        break
print(len(list3), len(list2))

717 2656


In [39]:
match_sub = []
no_match = []
part_match = {}
part_match2 = {}
perfect = []
for j in sub_rent2 :
    match = False
    if j in list3:
        match =True
        match_sub.append(j)
        perfect.append(j)
    else: #j not in list2:
        for k in list3:
            if j in k:             
                #match_sub.append(j) 
                if k in part_match.keys():
                    part_match2[k]=j
                else:
                    part_match[k]=j
                match =True
                match_sub.append(k)
                #print(j+":"+k)
                
        if match==False:
            no_match.append(j)
        else:
            match_sub.append(j)
len(no_match)

23

In [40]:
kk = list(part_match.keys())
for k in kk:
    if k in perfect:
        del part_match[k] 
        print(k)

CARRUM DOWNS
MALVERN EAST
HEIDELBERG WEST


In [41]:
match_sub.append('MELBOURNE CITY')
match_sub.append('ST KILDA')
match_sub.append('MOUNT ELIZA')
match_sub.append('MOUNT MARTHA')
len(match_sub)

327

In [12]:
match_sub2 = []
no_match2 = []
for j in sub_rent2 :
    match = False
    if j in suburbs:
        match =True
        match_sub2.append(j)
    else: #j not in list2:
        for k in suburbs:
            if j in k:             
                print(j+":"+k)
                #match_sub2.append(j) 
                #match =True
                break
        if match==False:
            no_match2.append(j)
len(no_match2)

HAMPTON:HAMPTON EAST
SUNSHINE:SUNSHINE NORTH


16

In [13]:
match_sub2.append('CBD')
match_sub2.append('MELBOURNE')

match_sub2.append('ST KILDA RD')
match_sub2.append('ST KILDA')

match_sub2.append('HAMPTON')
match_sub2.append('HAMPTON NORTH')

match_sub2.append('SUNSHINE')
match_sub2.append('SUNSHINE WEST')

#match_sub2.append('YARRA RANGES')
#match_sub2.append('')

match_sub2.append('MT ELIZA')
match_sub2.append('MOUNT ELIZA')

match_sub2.append('MT MARTHA')
match_sub2.append('MOUNT MARTHA')

In [14]:
vic_loc2 = vic_loc[vic_loc['suburb'].isin(match_sub2)]
vic_loc3 = vic_loc2.drop_duplicates(['suburb'], keep='first')
vic_loc3 = vic_loc3[['postcode','suburb','lat','lon']]
vic_loc3.shape

(205, 4)

In [15]:
match_dic2 = {'MELBOURNE': 'CBD', 
             'HAMPTON NORTH':'HAMPTON',
             'SUNSHINE WEST': 'SUNSHINE',
             'MOUNT ELIZA': 'MT ELIZA',
             'MOUNT MARTHA':'MT MARTHA'  }
vic_loc3 = vic_loc3.replace({'suburb':match_dic2})

In [42]:
df3 = df2[df2['LABEL'].isin(match_sub)]
df4 = df3[['LABEL','Accommodation & Food Services','Health Care & Social Assistance','Total number of businesses']]
df4 = df4[~df4['Total number of businesses'].isna()]

In [43]:
df5 = df4.drop_duplicates(['LABEL'], keep='last')
df5 = df5.reset_index(drop=True)
df5.shape

(227, 4)

In [44]:
L = df5.shape[0]
for i in part_match2.keys():
    for j in range(L):
        if df5.loc[j,'LABEL'] == i:
            row = [part_match2[i]] + list(df5.iloc[j,1:].values) 
            df5 = df5.append(pd.Series(row, index=df5.columns ), ignore_index=True)
df5.shape

(233, 4)

In [45]:
match_dic = part_match.copy() 
match_dic.update({'MELBOURNE CITY': 'CBD', 
             'MOUNT ELIZA': 'MT ELIZA',
             'MOUNT MARTHA':'MT MARTHA'  })

In [46]:
df5 = df5.replace({'LABEL':match_dic})

In [47]:
df5.shape

(233, 4)

In [48]:
food = df5.groupby(['LABEL'])['Accommodation & Food Services'].sum().reset_index(name="Accommodation & Food Services")
health = df5.groupby(['LABEL'])['Health Care & Social Assistance'].sum().reset_index(name="Health Care & Social Assistance")
business = df5.groupby(['LABEL'])['Total number of businesses'].sum().reset_index(name="Total number of businesses")

In [49]:
industry = pd.merge(food, business, on=['LABEL', 'LABEL'])
industry1 = industry.copy()  #industry1 = pd.merge(industry, health, on=['LABEL', 'LABEL'])
result = pd.merge(vic_loc3, industry1, how='left', left_on=['suburb'], right_on=['LABEL'])

In [50]:
result1 = result.drop(['LABEL'], axis=1)
result1.shape

(205, 6)

In [24]:
#result2 = result1[~result1['Total number of businesses'].isna()]
result2 = result1.copy()
result2['log_conv'] = round(np.log(result2['Total number of businesses']),1)
result2['log_food'] = round(np.log(result2['Accommodation & Food Services']),1)

In [54]:
result2['convenient'] = pd.cut(result2['log_conv'],  [0, 6.4, 7.1, 7.7, 9,15],labels=[1, 2, 3,4,5])
result2['food_service'] = pd.cut(result2['log_food'],  [0, 3,3.9, 4.7, 7,15],labels=[1, 2, 3,4,5])

In [55]:
result3= result2[['postcode','suburb','lat','lon','Accommodation & Food Services','Total number of businesses','convenient','food_service']]
result3.columns = ['postcode','Suburb','lat','lon','Accommodation_Food Services','Total_businesses','convenient_level','food_service_level']

In [58]:
# fill na with the middle level
result3['convenient_level'] = np.where(result3['convenient_level'].isna(),3,result3['convenient_level'] )
result3['food_service_level'] = np.where(result3['food_service_level'].isna(),3,result3['food_service_level'] )

C:\Users\YHChen\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\YHChen\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [59]:
rent_final = pd.read_csv('rent_final.csv', low_memory=False)
rent_final1 = rent_final[['Region','Suburb','Average_per_person']]
rent_final2 = rent_final1.drop_duplicates(['Suburb'], keep='last')
rent_final2 = rent_final2.reset_index(drop=True)

In [60]:
result_new = pd.merge(result3, rent_final2 , how='left', left_on=['Suburb'], right_on=['Suburb'])

In [61]:
result_final = result_new[~(result_new['Average_per_person'].isna())]

In [62]:
result_final.shape

(204, 10)

In [63]:
result_final.to_csv('suburb_detail_final_v2.csv',index=False)

In [2]:
sub = pd.read_csv('suburb_detail_final_v2.csv', low_memory=False)

In [4]:
uni = pd.read_csv('campus_loc_v2.csv', low_memory=False)

In [9]:
uni['Suburb'] = uni['Suburb'].str.upper()
sub2 = sub[['Suburb','Average_per_person']]

In [10]:
uni1 = pd.merge(uni, sub2 , how='left', left_on=['Suburb'], right_on=['Suburb'])

In [11]:
uni1.head()

,uni_id,University,Campus,Suburb,lat,long,Average_per_person
0,1,La Trobe University,Melboune,BUNDOORA,-37.7170,145.0432,178
1,2,Monash University,Clayton,CLAYTON,-37.9153,145.1340,191
2,3,RMIT University,City,CBD,-37.8081,144.9633,294
3,4,Swinburne University of Technology,Hawthorn,HAWTHORN,-37.8529,144.9917,225
4,5,University of Divinity,Melboune,KEW,-37.8057,145.0368,212


In [12]:
uni1.to_csv('campus_loc_v3.csv', index=False)

In [3]:
sub.head()

,postcode,Suburb,lat,lon,Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person
0,3000,CBD,-37.814563,144.970267,2125.0,37323.0,5,5,Inner Melbourne,294
1,3002,EAST MELBOURNE,-37.816640,144.987811,48.0,1433.0,3,2,Inner Melbourne,288
2,3003,WEST MELBOURNE,-37.806255,144.941123,5.0,171.0,1,1,Inner Melbourne,260
3,3006,SOUTHBANK,-37.823258,144.965926,137.0,2751.0,4,4,Inner Melbourne,289
4,3008,DOCKLANDS,-37.814719,144.948039,148.0,2004.0,3,4,Inner Melbourne,308


In [4]:
house = pd.read_csv('rent_final_house_modified.csv', low_memory=False)

In [5]:
house.head()

,Rent_id,Region,Suburb,Average_per_person
0,0,Inner Melbourne,ALBERT PARK,313
1,1,Inner Melbourne,MIDDLE PARK,313
2,2,Inner Melbourne,WEST ST KILDA,313
3,3,Inner Melbourne,ARMADALE,304
4,4,Inner Melbourne,CARLTON NORTH,249


In [8]:
sub[sub['Suburb']=="WODONGA"]

,postcode,Suburb,lat,lon,Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person
181,3689,WODONGA,-37.67576,144.989793,73.0,2000.0,3,3,Other Regional Centres,116


In [6]:
house1 = house[['Suburb','Average_per_person']]

In [7]:
house1.columns = ['Suburb', 'House_rent']

In [8]:
house1.head()

,Suburb,House_rent
0,ALBERT PARK,313
1,MIDDLE PARK,313
2,WEST ST KILDA,313
3,ARMADALE,304
4,CARLTON NORTH,249


In [9]:
house2 = pd.merge(sub, house1 , how='left', left_on=['Suburb'], right_on=['Suburb'])

In [10]:
house2.head()

,postcode,Suburb,lat,lon,Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person,House_rent
0,3000,CBD,-37.814563,144.970267,2125.0,37323.0,5,5,Inner Melbourne,294,NaN
1,3002,EAST MELBOURNE,-37.816640,144.987811,48.0,1433.0,3,2,Inner Melbourne,288,NaN
2,3003,WEST MELBOURNE,-37.806255,144.941123,5.0,171.0,1,1,Inner Melbourne,260,233.0
3,3006,SOUTHBANK,-37.823258,144.965926,137.0,2751.0,4,4,Inner Melbourne,289,NaN
4,3008,DOCKLANDS,-37.814719,144.948039,148.0,2004.0,3,4,Inner Melbourne,308,333.0


In [19]:
house2[house2['House_rent'].isnull()]

,postcode,Suburb,lat,lon,Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person,House_rent
0,3000,CBD,-37.814563,144.970267,2125.0,37323.0,5,5,Inner Melbourne,294,NaN
1,3002,EAST MELBOURNE,-37.816640,144.987811,48.0,1433.0,3,2,Inner Melbourne,288,NaN
3,3006,SOUTHBANK,-37.823258,144.965926,137.0,2751.0,4,4,Inner Melbourne,289,NaN


In [21]:
match_level = {1: 'Very Low', 
             2:'Low',
             3: 'Medium',
             4: 'High',
             5:'Very High'  }
house2 = house2.replace({'convenient_level':match_level})
house2.head()

,postcode,Suburb,lat,lon,Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person,House_rent
0,3000,CBD,-37.814563,144.970267,2125.0,37323.0,Very High,5,Inner Melbourne,294,NaN
1,3002,EAST MELBOURNE,-37.816640,144.987811,48.0,1433.0,Medium,2,Inner Melbourne,288,NaN
2,3003,WEST MELBOURNE,-37.806255,144.941123,5.0,171.0,Very Low,1,Inner Melbourne,260,350.0
3,3006,SOUTHBANK,-37.823258,144.965926,137.0,2751.0,High,4,Inner Melbourne,289,NaN
4,3008,DOCKLANDS,-37.814719,144.948039,148.0,2004.0,Medium,4,Inner Melbourne,308,333.0


In [22]:
house2 = house2.replace({'food_service_level':match_level})
house2.head()

,postcode,Suburb,lat,lon,Food Services,Total_businesses,convenient_level,food_service_level,Region,Average_per_person,House_rent
0,3000,CBD,-37.814563,144.970267,2125.0,37323.0,Very High,Very High,Inner Melbourne,294,NaN
1,3002,EAST MELBOURNE,-37.816640,144.987811,48.0,1433.0,Medium,Low,Inner Melbourne,288,NaN
2,3003,WEST MELBOURNE,-37.806255,144.941123,5.0,171.0,Very Low,Very Low,Inner Melbourne,260,350.0
3,3006,SOUTHBANK,-37.823258,144.965926,137.0,2751.0,High,High,Inner Melbourne,289,NaN
4,3008,DOCKLANDS,-37.814719,144.948039,148.0,2004.0,Medium,High,Inner Melbourne,308,333.0


In [11]:
house2.to_csv('suburb_detail_final_v3.csv',index=False)

In [ ]:
result2['convenient'] = pd.cut(result2['log_conv'],  [0, 6.4, 7.1, 7.7, 9,15],labels=[1, 2, 3,4,5])
result2['food_service'] = pd.cut(result2['log_food'],  [0, 3,3.9, 4.7, 7,15],labels=[1, 2, 3,4,5])

In [5]:
print(round(np.exp(6.4)), round(np.exp(7.1)),round(np.exp(7.7)),round(np.exp(9)))   

602.0 1212.0 2208.0 8103.0


In [6]:
print(round(np.exp(3)), round(np.exp(3.9)),round(np.exp(4.7)),round(np.exp(7)))  

20.0 49.0 110.0 1097.0
